Part B: Risk Scoring & Decision Engine

This notebook applies risk logic on top of the synthetic data generated in Part A.

Input:

Monthly synthetic transaction file

Customer trust state (already embedded in data)

Output:

Transaction-level risk score

Decision: Allow / Review / Block

Reason codes explaining the decision

Design principles:

Explainable logic

Layered risk (rules + ML)

Deterministic behaviour

Reviewer-friendly structure

This notebook can be re-run for any month independently.

In [1]:
# ============================================================
# PHASE B0: SETUP & CONFIGURATION
# ============================================================

import pandas as pd
import numpy as np
import os

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 200)


In [28]:
# ============================================================
# PHASE B0: LOAD MONTHLY DATA
# ============================================================

# ----------------------------
# Select month to analyse
# ----------------------------
ANALYSIS_MONTH = "2025_03"   # change to 2025_01, 2025_02, etc.

MONTHLY_DATA_PATH = rf"\CROSS_BORDER_FRAUD\INPUTS\TRANSACTION_OUTPUT_FOLDER\synthetic_transactions_{ANALYSIS_MONTH}.csv"

# ----------------------------
# Load data
# ----------------------------
df = pd.read_csv(
    MONTHLY_DATA_PATH,
    parse_dates=["transaction_timestamp"]
)

print("Loaded data shape:", df.shape)
display(df.head())


Loaded data shape: (106054, 11)


,transaction_id,customer_id,transaction_timestamp,source_country,destination_country,source_currency,destination_currency,transaction_amount,is_new_device,is_new_customer,trust_score
0,TXN_3234489_CUST_000001,CUST_000001,2025-03-29 14:00:00,IN,US,INR,USD,379.40,0,0,54
1,TXN_5472471_CUST_000001,CUST_000001,2025-03-19 22:00:00,IN,US,INR,USD,76.14,0,0,54
2,TXN_5521373_CUST_000001,CUST_000001,2025-03-03 21:00:00,IN,AE,INR,AED,75.39,0,0,54
3,TXN_6664789_CUST_000001,CUST_000001,2025-03-06 01:00:00,IN,UK,INR,GBP,131.82,0,0,54
4,TXN_5721339_CUST_000002,CUST_000002,2025-03-12 16:00:00,IN,UK,INR,GBP,286.15,0,0,32


Phase B1: Feature Engineering

In this phase, we transform raw transaction data into risk-relevant features.

Principles:

Features must be explainable

Features must map to real payment risk patterns

No “black-box” transformations

These features will be reused by:

Rule-based logic

ML model

Reason codes

In [29]:
# ============================================================
# PHASE B1: FEATURE ENGINEERING
# ============================================================

# Make a working copy
features_df = df.copy()

# ----------------------------
# Amount-based features
# ----------------------------
features_df["amount_log"] = np.log1p(features_df["transaction_amount"])

# High amount flag (relative threshold)
amount_95 = features_df["transaction_amount"].quantile(0.95)
features_df["is_high_amount"] = (features_df["transaction_amount"] > amount_95).astype(int)

# ----------------------------
# Time-based features
# ----------------------------
features_df["txn_hour"] = features_df["transaction_timestamp"].dt.hour
features_df["is_odd_hour"] = features_df["txn_hour"].isin(range(0, 5)).astype(int)

# ----------------------------
# Corridor risk feature
# ----------------------------
# Define corridor risk inline (policy driven)
CORRIDOR_RISK_SCORE = {
    ("IN", "US"): 0.2,
    ("IN", "UK"): 0.2,
    ("IN", "AE"): 0.3,
    ("IN", "SG"): 0.3,
    ("IN", "NG"): 0.9
}

features_df["corridor_risk"] = features_df.apply(
    lambda x: CORRIDOR_RISK_SCORE.get(
        (x["source_country"], x["destination_country"]), 0.5
    ),
    axis=1
)

# High-risk corridor flag
features_df["is_high_risk_corridor"] = (features_df["corridor_risk"] >= 0.7).astype(int)

# ----------------------------
# Customer trust context
# ----------------------------
features_df["is_low_trust"] = (features_df["trust_score"] < 40).astype(int)

# ----------------------------
# Select final feature set
# ----------------------------
FEATURE_COLUMNS = [
    "amount_log",
    "is_high_amount",
    "is_odd_hour",
    "is_new_device",
    "is_new_customer",
    "corridor_risk",
    "is_high_risk_corridor",
    "trust_score",
    "is_low_trust"
]

print("Feature columns created:")
print(FEATURE_COLUMNS)

display(features_df[FEATURE_COLUMNS + ["transaction_amount", "trust_score"]].head())


Feature columns created:
['amount_log', 'is_high_amount', 'is_odd_hour', 'is_new_device', 'is_new_customer', 'corridor_risk', 'is_high_risk_corridor', 'trust_score', 'is_low_trust']


,amount_log,is_high_amount,is_odd_hour,is_new_device,is_new_customer,corridor_risk,is_high_risk_corridor,trust_score,is_low_trust,transaction_amount,trust_score
0,5.941223,0,0,0,0,0.2,0,54,0,379.40,54
1,4.345622,0,0,0,0,0.2,0,54,0,76.14,54
2,4.335852,0,0,0,0,0.3,0,54,0,75.39,54
3,4.888995,0,1,0,0,0.2,0,54,0,131.82,54
4,5.660005,0,0,0,0,0.2,0,32,1,286.15,32


Phase B2: Rule-based Risk Signals

In this phase, we apply explicit risk rules that flag suspicious transactions.

Why rules:

Rules are explainable

Rules are easy to audit

Rules act as the first line of defence

Each rule produces:

a binary signal (0 / 1)

a clear semantic meaning

These rule signals will later be:

combined with ML score

used for final decisions

converted into reason codes

In [30]:
# ============================================================
# PHASE B2: RULE-BASED RISK SIGNALS
# ============================================================

rules_df = features_df.copy()

# ----------------------------
# Rule 1: High amount on high-risk corridor
# ----------------------------
rules_df["rule_high_amount_high_risk_corridor"] = (
    (rules_df["is_high_amount"] == 1) &
    (rules_df["is_high_risk_corridor"] == 1)
).astype(int)

# ----------------------------
# Rule 2: New device + odd hour
# ----------------------------
rules_df["rule_new_device_odd_hour"] = (
    (rules_df["is_new_device"] == 1) &
    (rules_df["is_odd_hour"] == 1)
).astype(int)

# ----------------------------
# Rule 3: Low trust customer on high-risk corridor
# ----------------------------
rules_df["rule_low_trust_high_risk_corridor"] = (
    (rules_df["is_low_trust"] == 1) &
    (rules_df["is_high_risk_corridor"] == 1)
).astype(int)

# ----------------------------
# Rule 4: New customer + high amount
# ----------------------------
rules_df["rule_new_customer_high_amount"] = (
    (rules_df["is_new_customer"] == 1) &
    (rules_df["is_high_amount"] == 1)
).astype(int)

# ----------------------------
# Aggregate rule signals
# ----------------------------
RULE_COLUMNS = [
    "rule_high_amount_high_risk_corridor",
    "rule_new_device_odd_hour",
    "rule_low_trust_high_risk_corridor",
    "rule_new_customer_high_amount"
]

rules_df["rule_trigger_count"] = rules_df[RULE_COLUMNS].sum(axis=1)

print("Rule columns created:")
print(RULE_COLUMNS)

#display(
#    rules_df[RULE_COLUMNS + ["rule_trigger_count"]]
#    .value_counts()
#    .head(10)
#)


Rule columns created:
['rule_high_amount_high_risk_corridor', 'rule_new_device_odd_hour', 'rule_low_trust_high_risk_corridor', 'rule_new_customer_high_amount']


In [31]:
rules_df.head(2)

,transaction_id,customer_id,transaction_timestamp,source_country,destination_country,source_currency,destination_currency,transaction_amount,is_new_device,is_new_customer,trust_score,amount_log,is_high_amount,txn_hour,is_odd_hour,corridor_risk,is_high_risk_corridor,is_low_trust,rule_high_amount_high_risk_corridor,rule_new_device_odd_hour,rule_low_trust_high_risk_corridor,rule_new_customer_high_amount,rule_trigger_count
0,TXN_3234489_CUST_000001,CUST_000001,2025-03-29 14:00:00,IN,US,INR,USD,379.40,0,0,54,5.941223,0,14,0,0.2,0,0,0,0,0,0,0
1,TXN_5472471_CUST_000001,CUST_000001,2025-03-19 22:00:00,IN,US,INR,USD,76.14,0,0,54,4.345622,0,22,0,0.2,0,0,0,0,0,0,0


In [32]:
rules_df.rule_trigger_count.value_counts()

rule_trigger_count
0    99476
1     6214
2      356
3        8
Name: count, dtype: int64

Phase B3: ML-based Risk Score

In this phase, we generate a machine-learning risk score for each transaction.

Key points:

This is a probabilistic score, not a decision

The model complements rule-based logic

The goal is ranking risk, not predicting fraud perfectly

Since this is synthetic data:

We create a proxy fraud label using high-risk patterns

The model learns from a combination of rules and features

Model choice:

Logistic Regression

Chosen for explainability and stability

In [33]:
# ============================================================
# PHASE B3: CREATE PROXY FRAUD LABEL
# ============================================================

ml_df = rules_df.copy()

# Proxy fraud label:
# Transactions triggering multiple strong signals are treated as risky
ml_df["fraud_label"] = (
    (ml_df["rule_trigger_count"] >= 2) |
    ((ml_df["rule_trigger_count"] == 1) & (ml_df["is_low_trust"] == 1))
).astype(int)

print("Fraud label distribution:")
print(ml_df["fraud_label"].value_counts(normalize=True))


Fraud label distribution:
fraud_label
0    0.96086
1    0.03914
Name: proportion, dtype: float64


In [34]:
# ============================================================
# PHASE B3: TRAIN ML MODEL & SCORE TRANSACTIONS
# ============================================================

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

ML_FEATURES = [
    "amount_log",
    "is_high_amount",
    "is_odd_hour",
    "is_new_device",
    "is_new_customer",
    "corridor_risk",
    "trust_score",
    "rule_trigger_count"
]

X = ml_df[ML_FEATURES]
y = ml_df["fraud_label"]

# Build explainable pipeline
ml_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=1000))
])

ml_pipeline.fit(X, y)

# Generate ML risk score (probability of fraud)
ml_df["ml_risk_score"] = ml_pipeline.predict_proba(X)[:, 1]

print("ML risk score summary:")
display(ml_df["ml_risk_score"].describe())


ML risk score summary:


count    1.060540e+05
mean     3.915825e-02
std      1.917007e-01
min      4.213812e-26
25%      7.554796e-20
50%      6.585440e-16
75%      3.941268e-12
max      1.000000e+00
Name: ml_risk_score, dtype: float64

Phase B4: Final Decision Logic

In this phase, we convert risk signals into business decisions.

Decision philosophy:

BLOCK only when confidence is high

REVIEW borderline transactions

ALLOW low-risk transactions automatically

Inputs to decision:

Rule trigger count

ML risk score

Customer trust score

Output:

One decision per transaction:

ALLOW

REVIEW

BLOCK

This mirrors real payment risk decision engines.

In [35]:
# ============================================================
# PHASE B4: FINAL DECISION LOGIC
# ============================================================

decision_df = ml_df.copy()

def assign_decision(row):
    # Hard blocks (very high confidence)
    if row["rule_trigger_count"] >= 3:
        return "BLOCK"

    if row["ml_risk_score"] >= 0.90 and row["trust_score"] < 40:
        return "BLOCK"

    # Review zone
    if row["ml_risk_score"] >= 0.60:
        return "REVIEW"

    if row["rule_trigger_count"] >= 1 and row["trust_score"] < 50:
        return "REVIEW"

    # Default allow
    return "ALLOW"


decision_df["decision"] = decision_df.apply(assign_decision, axis=1)

print("Decision distribution:")
display(decision_df["decision"].value_counts(normalize=True))


Decision distribution:


decision
ALLOW     0.955296
BLOCK     0.037330
REVIEW    0.007374
Name: proportion, dtype: float64

Phase B5: Reason Codes

In this phase, we generate human-readable reason codes for each transaction decision.

Why reason codes matter:

Required for audits and compliance

Used by operations teams during review

Help explain decisions to customers

Improve trust in automated systems

Design principles:

Reasons are derived from rules and signals

Multiple reasons can apply to one transaction

Reasons must be simple and clear

In [36]:
# ============================================================
# PHASE B5: GENERATE REASON CODES
# ============================================================

reason_df = decision_df.copy()

def generate_reason_codes(row):
    reasons = []

    if row["rule_high_amount_high_risk_corridor"] == 1:
        reasons.append("High amount on high-risk corridor")

    if row["rule_new_device_odd_hour"] == 1:
        reasons.append("New device used during odd hours")

    if row["rule_low_trust_high_risk_corridor"] == 1:
        reasons.append("Low trust customer on high-risk corridor")

    if row["rule_new_customer_high_amount"] == 1:
        reasons.append("New customer with high transaction amount")

    if row["ml_risk_score"] >= 0.90:
        reasons.append("Very high ML risk score")

    if row["ml_risk_score"] >= 0.60 and row["ml_risk_score"] < 0.90:
        reasons.append("Elevated ML risk score")

    if len(reasons) == 0:
        reasons.append("No significant risk signals")

    return reasons


reason_df["reason_codes"] = reason_df.apply(generate_reason_codes, axis=1)

# Convert list to readable string
reason_df["reason_codes_str"] = reason_df["reason_codes"].apply(lambda x: " | ".join(x))

display(
    reason_df[
        ["decision", "reason_codes_str", "ml_risk_score", "trust_score"]
    ].head(10)
)


,decision,reason_codes_str,ml_risk_score,trust_score
0,ALLOW,No significant risk signals,2.591736e-15,54
1,ALLOW,No significant risk signals,2.315353e-15,54
2,ALLOW,No significant risk signals,2.299584e-15,54
3,ALLOW,No significant risk signals,6.591601e-16,54
4,ALLOW,No significant risk signals,5.570386e-09,32
5,ALLOW,No significant risk signals,5.525715e-09,32
6,ALLOW,No significant risk signals,5.787659e-09,32
7,BLOCK,Low trust customer on high-risk corridor | Ver...,9.980558e-01,32
8,ALLOW,No significant risk signals,5.717055e-09,32
9,ALLOW,No significant risk signals,2.472247e-15,54


Phase B6: Final Output & Sanity Checks

In this final phase, we prepare the production-style output of the risk engine.

What this includes:

Transaction details

Risk signals

ML risk score

Final decision

Human-readable reason codes

We also run basic sanity checks to ensure:

Decisions are well distributed

Risk scores align with decisions

Output is ready for downstream use (dashboards, audits, reviews)

This dataset represents what a real payment risk system would output.

In [37]:
# ============================================================
# PHASE B6: FINAL OUTPUT & SANITY CHECKS
# ============================================================

# ----------------------------
# Select final output columns
# ----------------------------
FINAL_COLUMNS = [
    "transaction_id",
    "customer_id",
    "transaction_timestamp",
    "transaction_amount",
    "source_country",
    "destination_country",
    "is_new_device",
    "is_new_customer",
    "trust_score",
    "rule_trigger_count",
    "ml_risk_score",
    "decision",
    "reason_codes_str"
]

final_output_df = reason_df[FINAL_COLUMNS].copy()

# ----------------------------
# Sanity checks
# ----------------------------
print("Final output shape:", final_output_df.shape)

print("\nDecision distribution:")
display(final_output_df["decision"].value_counts(normalize=True))

print("\nAverage ML risk score by decision:")
display(
    final_output_df.groupby("decision")["ml_risk_score"].mean()
)

# ----------------------------
# Save final output
# ----------------------------
OUTPUT_FOLDER = r"\CROSS_BORDER_FRAUD\OUTPUTS\RISK_SCORE_TXNS"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

final_output_path = os.path.join(
    OUTPUT_FOLDER,
    f"risk_scored_transactions_{ANALYSIS_MONTH}.csv"
)

final_output_df.to_csv(final_output_path, index=False)

print("\nFinal risk-scored dataset saved to:")
print(final_output_path)

display(final_output_df.head())


Final output shape: (106054, 13)

Decision distribution:


decision
ALLOW     0.955296
BLOCK     0.037330
REVIEW    0.007374
Name: proportion, dtype: float64


Average ML risk score by decision:


decision
ALLOW     0.000005
BLOCK     0.994765
REVIEW    0.273793
Name: ml_risk_score, dtype: float64


Final risk-scored dataset saved to:
C:\Users\91833\Desktop\FRAUD_PROJECTS\CROSS_BORDER_FRAUD\OUTPUTS\RISK_SCORE_TXNS\risk_scored_transactions_2025_03.csv


,transaction_id,customer_id,transaction_timestamp,transaction_amount,source_country,destination_country,is_new_device,is_new_customer,trust_score,rule_trigger_count,ml_risk_score,decision,reason_codes_str
0,TXN_3234489_CUST_000001,CUST_000001,2025-03-29 14:00:00,379.40,IN,US,0,0,54,0,2.591736e-15,ALLOW,No significant risk signals
1,TXN_5472471_CUST_000001,CUST_000001,2025-03-19 22:00:00,76.14,IN,US,0,0,54,0,2.315353e-15,ALLOW,No significant risk signals
2,TXN_5521373_CUST_000001,CUST_000001,2025-03-03 21:00:00,75.39,IN,AE,0,0,54,0,2.299584e-15,ALLOW,No significant risk signals
3,TXN_6664789_CUST_000001,CUST_000001,2025-03-06 01:00:00,131.82,IN,UK,0,0,54,0,6.591601e-16,ALLOW,No significant risk signals
4,TXN_5721339_CUST_000002,CUST_000002,2025-03-12 16:00:00,286.15,IN,UK,0,0,32,0,5.570386e-09,ALLOW,No significant risk signals
